# Анализ крупного сервиса для чтения книг по подписке

**Коронавирус застал мир врасплох, изменив привычный порядок вещей. На какое-то время жители городов перестали выходить на улицу, посещать кафе и торговые центры. Зато стало больше времени для книг. Это заметили стартаперы — и бросились создавать приложения для тех, кто любит читать.**

**Ваша компания решила быть на волне и купила крупный сервис для чтения книг по подписке. Ваша первая задача как аналитика — проанализировать базу данных.**

**В ней — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. Эти данные помогут сформулировать ценностное предложение для нового продукта.**


# Задания:

- **Посчитайте, сколько книг вышло после 1 января 2000 года;**


- **Для каждой книги посчитайте количество обзоров и среднюю оценку;**


- **Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры;**


- **Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками;**


- **Посчитайте среднее количество обзоров от пользователей, которые поставили больше 48 оценок.**

## Вывод первых строк таблиц

# books:

In [2]:
books = '''
SELECT *
FROM books
LIMIT 10
'''

pd.io.sql.read_sql(sql=text(books), con = con)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268
5,6,257,1st to Die (Women's Murder Club #1),424,2005-05-20,116
6,7,258,2nd Chance (Women's Murder Club #2),400,2005-05-20,116
7,8,260,4th of July (Women's Murder Club #4),448,2006-06-01,318
8,9,563,A Beautiful Mind,461,2002-02-04,104
9,10,445,A Bend in the Road,341,2005-04-01,116


# authors:

In [3]:
authors = '''
SELECT *
FROM authors
LIMIT 10
'''

pd.io.sql.read_sql(sql=text(authors), con = con)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd
5,6,Alan Paton
6,7,Albert Camus/Justin O'Brien
7,8,Aldous Huxley
8,9,Aldous Huxley/Christopher Hitchens
9,10,Aleksandr Solzhenitsyn/H.T. Willetts


# publishers:

In [4]:
publishers = '''
SELECT *
FROM publishers
LIMIT 10
'''

pd.io.sql.read_sql(sql=text(publishers), con = con)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company
5,6,Aladdin
6,7,Aladdin Paperbacks
7,8,Albin Michel
8,9,Alfred A. Knopf
9,10,Alfred A. Knopf Books for Young Readers


# ratings:

In [5]:
ratings = '''
SELECT *
FROM ratings
LIMIT 10
'''

pd.io.sql.read_sql(sql=text(ratings), con = con)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2
5,6,3,johnsonamanda,4
6,7,3,scotttamara,5
7,8,3,lesliegibbs,5
8,9,4,abbottjames,5
9,10,4,valenciaanne,4


# reviews:

In [6]:
reviews = '''
SELECT *
FROM reviews
LIMIT 10
'''

pd.io.sql.read_sql(sql=text(reviews), con = con)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...
5,6,3,lesliegibbs,Analysis no several cause international.
6,7,4,valenciaanne,One there cost another. Say type save. With pe...
7,8,4,abbottjames,Within enough mother. There at system full rec...
8,9,5,npowers,Thank now focus realize economy focus fly. Ite...
9,10,5,staylor,Game push lot reduce where remember. Including...


## Посчитайте, сколько книг вышло после 1 января 2000 года

In [7]:
books_after_2000 = '''
SELECT COUNT(book_id)
FROM books
WHERE publication_date >= '2000-01-01'
'''

pd.io.sql.read_sql(sql=text(books_after_2000), con = con)

,count
0,821


**После 1 января 2000 года вышла 821 книга.**

## Для каждой книги посчитайте количество обзоров и среднюю оценку

In [8]:
books_reviews_ratings = '''
WITH
reviews_count AS (SELECT book_id,
                  COUNT(review_id) AS reviews_count
                  FROM reviews
                  GROUP BY book_id),
ratings_avg AS (SELECT book_id,
                ROUND(AVG(rating), 2) AS avg_rating
                FROM ratings
                GROUP BY book_id)
                                  
SELECT b.title,
       COALESCE(rev_c.reviews_count, 0) AS reviews_count,
       rat_a.avg_rating
FROM books AS b
LEFT JOIN reviews_count AS rev_c ON b.book_id = rev_c.book_id
LEFT JOIN ratings_avg AS rat_a ON b.book_id = rat_a.book_id
ORDER BY reviews_count DESC, rat_a.avg_rating DESC
'''

pd.io.sql.read_sql(sql=text(books_reviews_ratings), con = con)

,title,reviews_count,avg_rating
0,Twilight (Twilight #1),7,3.66
1,Harry Potter and the Prisoner of Azkaban (Harr...,6,4.41
2,Harry Potter and the Chamber of Secrets (Harry...,6,4.29
3,The Book Thief,6,4.26
4,The Glass Castle,6,4.21
...,...,...,...
995,Leonardo's Notebooks,0,4.00
996,Disney's Beauty and the Beast (A Little Golden...,0,4.00
997,Essential Tales and Poems,0,4.00
998,Anne Rice's The Vampire Lestat: A Graphic Novel,0,3.67


**Книга "Twilight (Twilight #1)" имеет самое большое кол-во обзоров.**

## Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры

In [9]:
publisher_a_lot = '''
SELECT p.publisher,
       COUNT(b.book_id) AS book_count
FROM publishers AS p
JOIN books AS b ON p.publisher_id = b.publisher_id
WHERE b.num_pages > 50
GROUP BY p.publisher
ORDER BY book_count DESC
LIMIT 1
'''

pd.io.sql.read_sql(sql=text(publisher_a_lot), con = con)

,publisher,book_count
0,Penguin Books,42


**Издательство "Penguin Books" выпустило наибольшее число книг, не считая брошюры.**

## Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками

In [10]:
author_high_rating = '''
WITH
books_rating AS (SELECT b.book_id,
                        b.author_id,
                        COUNT(r.rating_id) AS rating_count,
                        AVG(r.rating) AS avg_rating
                 FROM books AS b
                 JOIN ratings AS r ON b.book_id = r.book_id
                 GROUP BY b.book_id
                 HAVING COUNT(r.rating_id) >= 50)

SELECT a.author,
       ROUND(AVG(b_r.avg_rating), 2) AS author_rating
FROM books_rating AS b_r
JOIN authors AS a ON b_r.author_id = a.author_id
GROUP BY a.author_id
ORDER BY author_rating DESC
LIMIT 1
'''

pd.io.sql.read_sql(sql=text(author_high_rating), con = con)

,author,author_rating
0,J.K. Rowling/Mary GrandPré,4.28


**Авторы J.K. Rowling и Mary GrandPré имеют самый высокий средний рейтинг по своим книгам.**

## Посчитайте среднее количество обзоров от пользователей, которые поставили больше 48 оценок

In [11]:
mean_reviews = '''
WITH
reviews_count AS (SELECT COUNT(review_id) AS reviews_count
                  FROM reviews
                  WHERE username IN (SELECT rat.username
                                     FROM ratings AS rat
                                     JOIN books AS b ON rat.book_id = b.book_id
                                     GROUP BY rat.username
                                     HAVING COUNT(rat.rating_id) > 48)
                  GROUP BY username)

SELECT AVG(reviews_count)::int AS avg_reviews
FROM reviews_count
'''

pd.io.sql.read_sql(sql=text(mean_reviews), con = con)

,avg_reviews
0,24


**Пользователи, которые ставят больше 48 оценок - имеют в среднем 24 обзора.**

## Вывод по исследованию

- **После 1 января 2000 года вышла 821 книга;**


- **Книга "Twilight (Twilight #1)" имеет самое большое кол-во обзоров;**


- **Издательство "Penguin Books" выпустило наибольшее число книг, не считая брошюры;**


- **Авторы J.K. Rowling и Mary GrandPré имеют самый высокий средний рейтинг по своим книгам;**


- **Пользователи, которые ставят больше 48 оценок - имеют в среднем 24 обзора.**